In [1]:
%load_ext autoreload
%autoreload 2

from llama3.eval import Experiment
from llama3.utils.load_utils import load_config, save_config, setup_log_dir

config = load_config("configs/bayes_eval.yaml")

experiment_log_dir = setup_log_dir(
    config.get("logs_dir", "experiment_logs"), experiment_name=config.get("experiment_name", None)
)

config["experiment_config"]["experiment_log_dir"] = experiment_log_dir
save_config(config.to_dict(), experiment_log_dir + "/config.yaml")

experiment = Experiment(config["experiment_config"])

/home/paperspace/miniconda/envs/posteriors/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.03it/s]
Some weights of BayesLlamaForCausalLM were not initialized from the model checkpoint at Meta-Llama-3-8B and are newly initialized: ['model.bayesian_layers.0.input_layernorm.weight', 'model.bayesian_layers.0.mlp.down_proj.weight', 'model.bayesian_layers.0.mlp.gate_proj.weight', 'model.bayesian_layers.0.mlp.up_proj.weight', 'model.bayesian_layers.0.post_attention_layernorm.weight', 'model.bayesian_layers.0.self_attn.k_proj.weight', 'model.bayesian_layers.0.self_attn.o_proj.weight', 'model.bayesian_layers.0.self_attn.q_proj.weigh

In [14]:
experiment.chat_model=False
experiment.run(config["dataset_path"][0])

  0%|          | 0/210 [00:00<?, ?it/s]

Answer the following multiple choice question very succintly.

Steps of the scientific method include all of the following except
a. doing background research.
b. constructing a hypothesis.
c. asking a question.
d. proving a theory.

Answer:
Answer the following multiple choice question very succintly.

Why do scientists call the Big Bang a theory?
a. It is probably unlikely and therefore not a fact.
b. A very well respected scientist proved it to be true.
c. Many scientists have agreed upon this explanation after repeated experiments and models have shown it
d. All possible answers to a scientific idea are called theories.

Answer:
Answer the following multiple choice question very succintly.

The data collected in an experiment should always be
a. labeled.
b. recorded.
c. reported.
d. all of the above

Answer:
Answer the following multiple choice question very succintly.

Which of the following is not a scientific model?
a. A cross section of an apple that mimics the layers of the Ea

  0%|          | 0/210 [00:01<?, ?it/s]

[' The correct option is (', ' The Big Bang is called a theory because many scientists', ' d', ' The correct answer is option b. A chart with', ' d']


{}

In [11]:
import torch

inputs_a = """
Answer the following multiple choice question as succintly as possible.

Steps of the scientific method include all of the following except
a. doing background research.
b. constructing a hypothesis.
c. asking a question.
d. proving a theory.

Answer:
"""
inputs_b = """
Answer the following multiple choice question as succintly as possible.

Why do scientists call the Big Bang a theory?
a. It is probably unlikely and therefore not a fact.
b. A very well respected scientist proved it to be true.
c. Many scientists have agreed upon this explanation after repeated experiments and models have shown it
d. All possible answers to a scientific idea are called theories.

Answer:
"""
inputs_c = """
Answer the following multiple choice question as succintly as possible.

The data collected in an experiment should always be
a. labeled.
b. recorded.
c. reported.
d. all of the above

Answer:
"""
inputs_d = """
Answer the following multiple choice question as succintly as possible.

If the results of an experiment disprove a hypothesis, then the
a. results should not be reported.
b. hypothesis is just a theory.
c. data must contain errors.
d. none of the above

Answer:
"""

@torch.no_grad()
def generate(model, inputs, max_length=20, use_cache=True):
    seq_out = []
    for idx in range(max_length):
        outputs = model(**inputs, return_dict=False, use_cache=use_cache)

        if "attention_mask" in inputs:
            del inputs["attention_mask"]

        next_token = outputs[0][0][:, -1].argmax(-1).unsqueeze(-1)
        if use_cache:
            inputs["past_key_values"] = outputs[1]
            inputs["ensemble_past_key_values"] = outputs[2]
            inputs["input_ids"] = next_token
        else:
            inputs["input_ids"] = torch.cat([inputs["input_ids"], next_token], dim=1)
        seq_out.append(next_token)

    return torch.cat(seq_out, -1)

inputs = [inputs_a, inputs_b, inputs_c, inputs_d]
inputs = experiment.tokenizer(inputs, padding=True, return_tensors="pt")
seq_out = generate(experiment.model, inputs.to("cuda"), use_cache=True)
experiment.tokenizer.batch_decode(seq_out, skip_special_tokens=True)

['The correct option isFigure ',
 'The correct option is d. All possible answers to a scientific idea are called theories.  The Big',
 'd',
 'd.  ://://Figure  Figure ']